<a href="https://colab.research.google.com/github/h5ng/GNN/blob/master/gpt_gnn_preprocess_OAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers texttable

In [2]:
from google.colab import drive
drive.mount('/content/dirve', force_remount=True)

gpt_gnn_path = '/content/dirve/MyDrive/GPT-GNN-master/example_OAG/GPT_GNN'
               
import sys
sys.path.append(gpt_gnn_path)

for p in sys.path:
  print(p)

from data import *

Mounted at /content/dirve

/content
/env/python
/usr/lib/python37.zip
/usr/lib/python3.7
/usr/lib/python3.7/lib-dynload
/usr/local/lib/python3.7/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.7/dist-packages/IPython/extensions
/root/.ipython
/content/dirve/MyDrive/GPT-GNN-master/example_OAG/GPT_GNN


In [3]:
from transformers import *
import gensim
from gensim.models import Word2Vec
from tqdm import tqdm

In [ ]:
args = {
  'input_dir': './data/oag_raw',
  'output_idr': './data/oag_output',
  'cuda': 0,
  'domain': '_CS',
  'citation_bar': 1
}

test_time_bar = 2016